In [2]:
#r "nuget: Shioaji, 1.1.8"

Installed Packages Shioaji, 1.1.8

In [3]:
using System.IO;
using Sinopac.Shioaji;
using System.Text.Json;
Shioaji _api = new Shioaji();

string jsonString = File.ReadAllText(@"D:\DotnetReactShioaji\DotnetReactShioaji\Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());

// var res = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]}) //若誤用[0]就導它無法在array裡搜，硬用linq就報錯

Connecting as shioaji@sinopac on 203.66.91.161:80...


In [4]:
// struct basicData
// {
//     public string code;
//     public string direction;
//     public double entry_price;
//     public double cover_price;
//     public double pnl;
// }

public dynamic anonListProfitLossSummary()
{
    var src = _api.ListProfitLossSummary("2024-01-18", "2024-01-18", _api.FutureAccount).profitloss_summary[0];
    var productAnonymous = new
    {
        code1 = (src.code.Substring(3, 5)) + (src.code[8] < 'L' ? "Call" : "Put"),
        direction = src.direction,
        entry_price = src.entry_price,
        cover_price = src.cover_price,
        netPnl = src.pnl - src.fee - src.tax
    };
    return productAnonymous;
}
anonListProfitLossSummary()

code1,17250Call
direction,Buy
entry_price,99.0
cover_price,125.0
netPnl,1259.0


In [ ]:
// System.Collections.Generic.  List<Sinopac.Shioaji.FutureProfitLossSummary>
// 匿名型別的變數有著direction, tax..屬性，而屬性被賦值的動作叫object initializer syntax
List<FutureProfitLossSummary> a = _api.ListProfitLossSummary("2024-01-18", "2024-01-18", _api.FutureAccount).profitloss_summary;
foreach(var i in a)  Console.WriteLine(i);

a
.Where(x => x.direction == "Buy")
// .Where(x => x.tax >= 0)
// .Where(x => x.fee >= 0)
.Where(x => x.code.Contains("TX4"))
.Where(x => x.quantity >= 1)
// .Where(x => x.entry_price >= 0)
// .Where(x => x.cover_price <= 999)
// .Where(x => x.pnl >= 0)
// .Select(x => x.tax)

In [ ]:
List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
var Pnl = _src.Where(x => x.code.Contains("MXF")).Select(x => x.pnl).FirstOrDefault();
var entry_price = _src.Where(x => x.code.Contains("MXF")).Select(x => x.price).FirstOrDefault();
decimal stpPt = entry_price - 25;

In [ ]:
List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
var Pnl = _src.Where(x => x.code.Contains("MXF")).Select(x => x.pnl).FirstOrDefault();
var entry_price = _src.Where(x => x.code.Contains("MXF")).Select(x => x.price).FirstOrDefault();
double stpPt = (double)entry_price - 25;
double spot = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]})[0].close;

List<double> _temp = new List<double>(); 
// 沒停損or沒收盤就迴圈
while(true)
{
    List<Sinopac.Shioaji.FuturePosition> _src_ = _api.ListPositions(_api.FutureAccount);
    if(_src_.Where(x => x.code.Contains("MXF")).Select(x => x.pnl).FirstOrDefault() >= -1250  
      ||  DateTime.Now > new DateTime(2024,1,25,5,0,0))
    {
        Console.WriteLine($"現在{spot}點，未實現：{Pnl}"); //監看若放成變數而變數沒持續更新，那迴圈就hang住了
    }
    else
    {
        //建容器一直存迄今高點, 若曾漲超過成本+12就把break條件設成保本
        _temp.Add(spot);
    }
    if(_temp.Max() > (double)entry_price + 12) stpPt = (double)entry_price + 2 ;
    if(spot < (double)stpPt) break;

}
Console.WriteLine($"結束。現在{spot}點，mxf未實現：{Pnl} 預期該是0或empty但linq不曉得怎麼防呆");
Console.WriteLine(_api.Snapshots(new List<IContract>() {_api.Contracts.Futures["QFF"]["QFFR1"]})[0].close);

// int cnt = 0;
// using System.Threading;
// if(true)
// {
//     double spot = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]})[0].close;
//     Console.WriteLine($"現在台指期{spot}，倒數{5 - cnt}次就停止");
//     Thread.Sleep(3000);
//     cnt += 1 ;
//     // if(cnt >= 5) 
//     if(_api.Snapshots(new List<IContract>() {_api.Contracts.Futures["QFF"]["QFFR1"]})[0].close <= 631
//     || cnt >= 5
//     )
//     break;    
// }

In [ ]:
// 1/16, 1/18, 1/24   entryprice若null就噴錯導致不能查多日
// _api.ListProfitLossSummary("2024-01-18", "2024-01-18", _api.FutureAccount)
// _api.ListProfitLossSummary("2024-01-18", "2024-01-18", _api.FutureAccount).profitloss_summary
_api.ListProfitLossSummary("2024-01-18", "2024-01-18", _api.FutureAccount).total